In [ ]:
def preProcess(s):
    stemmer = PorterStemmer()
    s = stemmer.stem(s) # stemming
    s = ' '.join(word for word in s.split() if len(word)>4) # remove words length less than 4
    return s

In [ ]:
vectorizer = TfidfVectorizer(stop_words = final_stopword_list, # this is a customized list
                             preprocessor=preProcess, 
                             max_df = 0.8,
                             min_df = 0.02, # the term must be in at least 20% of the document.
                             ngram_range=(1,3)) # unigram to trigrams will be presented as features

X = vectorizer.fit_transform(corpus1)
feature_names = vectorizer.get_feature_names()
print(feature_names)
print(len(vectorizer.get_feature_names()))
print(X.shape)

In [ ]:
svd = TruncatedSVD(60) # will include 60 major components
normalizer = Normalizer(copy = False) # Normalize samples individually to unit norm.
lsa = make_pipeline(svd, normalizer)
X = lsa.fit_transform(X)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

In [ ]:
# print the model equation
" ".join([
    "%+0.3f*%s" % (coef,feat)
    for coef, feat in zip(svd.components_[0],feature_names)
])

In [ ]:
#print the top 10 features
best_features = [feature_names[i] for i in svd.components_[0].argsort()[::-1]]
best_features[:10]

In [ ]:
true_k = 4 # you can try different k
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1, random_state = 1234)
model.fit(X)

In [ ]:
# print the top terms per cluster
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % feature_names[ind]),
    print

print("\n")
print(model.inertia_)

In [ ]:
# print the count per cluster
labels = model.predict(X)
freq =Counter(labels)
freq